In [ ]:
from typing import Dict, Any

import math            as math
import numpy           as np
import numpy.typing    as npt

#import plons scripts
import plons.PhysicalQuantities       as pq
import plons.ConversionFactors_cgs    as cgs
import plons.GeometricalFunctions     as gf

In [ ]:
from dotenv import load_dotenv
import os
from elasticsearch import helpers  # For bulk Data Uploading
from elasticsearch import Elasticsearch  # Base function for interacting with Elasticsearch

load_dotenv()
client = Elasticsearch("http://localhost:9200/", api_key=os.getenv('apikey'))

#test client
print(client.info())

In [ ]:
'''
"settings" is technically not needed if we are working on a simple local host, but can be changed to optimise search performance on a database that is hosted on a cluster and searched by multiple users.
"mappings" is required if you wish to explicitly map fields to specific values
'''

index_definition = {
    "settings": {  
        "number_of_shards": 1,
    },
    "mappings": {
        "properties": {
            "code hash": {"type": "keyword"},
            "path to folder": {"type": "keyword"},
            "simulation time": {
                "type": "float",
                "meta": {
                    "unit": "s",
                }
            },
            "particle mass": {
                "type": "float",
                "meta": {
                    "unit": "Msun",
                }
            },
            "tmax": {"type": "float"},
            "eccentricity": {"type": "float"},
            "semi-major axis": {
                "type": "float",
                "meta": {
                    "unit": "s",
                }
            },
            "wind mass rate": {
                "type": "float",
                "meta": {
                    "unit": "Msun/yr",
                }
            },
            "init wind velocity": {
                "type": "float",
                "meta": {
                    "unit": "km/s",
                }
            },
            "term wind velocity": {
                "type": "float",
                "meta": {
                    "unit": "km/s",
                }
            },
            "wind inject radius": {
                "type": "float",
                "meta": {
                    "unit": "au",
                }
            },
            "wind temperature": {
                "type": "float",
                "meta": {
                    "unit": "K",
                }
            },
        }
    }
}


### Load information from .setup file
#### List of Labels from Excel:
eccentricity	float		    .setup 

semi-major axis	float	au	    .setup

primary mass	float	Msol	.setup

primary Racc	float	Rsol	.setup

primary Reff	float	Rsol	.setup

primary Teff	float	K	    .setup

secondary mass	float	Msol	.setup

secondary Racc	float	Rsol	.setup

secondary Reff	float	Rsol	.setup

secondary Teff	float	K	    .setup

icompanion_star	integer		    .setup

subst	        interger		.setup

tertiary mass	float	Msol	.setup

tertiary Racc	float	Rsol	.setup

tertiary Reff	float	Rsol	.setup

tertiary Teff	float	K	    .setup

wind polytropic index	float	.setup



In [ ]:
def LoadSetup(dir: str, prefix: str) -> Dict[str, Any]:
    """ Load the prefix.in and prefix.setup files to get general information about the phantom model

    Args:
        dir (str): directory of the simulation
        prefix (str): prefix used for the files

    Returns:
        Dict[str, Any]: a dictionary containing the info from the setup files
        (!! check units, they are not all in SI or cgs)
    """

    # load the prefix.in & prefix.setup file
    setup = {}
    try:
        with open(os.path.join(dir,'%s.setup'%prefix), 'r') as f:
            lines = f.readlines()
            for string in lines:
                line = string.split()
                if len(line) != 0:
                    if line[0] != '#':
                        stringName = line[0]

                        # Floats
                        if stringName == 'primary_mass': stringName = 'massAGB_ini'
                        elif stringName == 'secondary_mass': stringName = 'massComp_ini'
                        elif stringName == 'semi_major_axis': stringName = 'sma_ini'
                        elif stringName == 'binary2_a' : stringName = 'sma_in_ini'
                        elif stringName == 'wind_gamma' or stringName == "temp_exponent": stringName = 'gamma'
                        elif stringName == 'eccentricity': stringName = 'ecc'
                        elif stringName == 'binary2_e' : stringName = 'ecc_in'
                        elif stringName == 'secondary_racc': stringName = 'rAccrComp'
                        elif stringName == 'accr2b' : stringName = 'rAccrComp_in'
                        elif stringName == 'racc2b' : stringName = 'rAccrComp_in'

                        setup[stringName] = float(line[2])

                        if stringName == 'q2':
                            stringName = 'massComp_in_ini'
                            setup[stringName] = float(line[2])*setup['massAGB_ini']

                        # Boolean
                        if stringName == 'icompanion_star':
                            stringName = 'single_star'
                            if int(line[2]) == 0: setup[stringName] = True
                            else: setup[stringName] = False
                            stringName = 'triple_star'
                            if int(line[2]) == 2: setup[stringName] = True
                            else: setup[stringName] = False

    except FileNotFoundError:
        print('')
        print(" ERROR: No %s.setup file found!"%prefix)
        print('')
        exit()

    try:
        with open(os.path.join(dir,'%s.in'%prefix), 'r') as f:
            lines = f.readlines()
            for string in lines:
                line = string.split()
                if len(line) != 0:
                    if line[0] != '#':
                        stringName = line[0]

                        # Strings
                        if stringName == 'logfile': setup[stringName] = str(line[2])
                        elif stringName == 'dumpfile': setup[stringName] = str(line[2])
                        elif stringName == 'twallmax': setup[stringName] = str(line[2])
                        elif stringName == 'dtwallmax': setup[stringName] = str(line[2])

                        # Floats
                        else:
                            if stringName == 'wind_velocity': stringName = 'v_ini'
                            elif stringName == 'outer_boundary': stringName = 'bound'
                            elif stringName == 'wind_mass_rate': stringName = 'Mdot'
                            if line[2]=='F': line[2] = 0
                            setup[stringName] = float(line[2])

    except FileNotFoundError:
        print('')
        print(" ERROR: No %s.in file found!"%prefix)
        print('')
        exit()

    
    # Additional Parameters
    massAGB_ini = setup["massAGB_ini"]
    v_ini = setup["v_ini"]
    if setup["single_star"] == False:
        massComp_ini = setup["massComp_ini"]
        sma = setup["sma_ini"]
        period = pq.getPeriod(massAGB_ini * cgs.Msun, massComp_ini * cgs.Msun, sma)           # [s]
        #v_orb = pq.getOrbitalVelocity(period, sma) * cgs.cms_kms()                           # [km/s]
        Rcap = pq.getCaptureRadius(massComp_ini * cgs.Msun, v_ini * cgs.kms) / cgs.au         # [au]
        if setup['triple_star']==True:
            massComp_in_ini = setup["massComp_in_ini"]
            sma_in = setup["sma_in_ini"]
            period = pq.getPeriod((massAGB_ini+massComp_in_ini) * cgs.Msun, massComp_ini * cgs.Msun, sma)  # [s]
            period_in = pq.getPeriod(massAGB_ini * cgs.Msun, massComp_in_ini * cgs.Msun, sma_in)           # [s]
            Rcap_in = pq.getCaptureRadius(massComp_in_ini * cgs.Msun, v_ini * cgs.kms) / cgs.au            # [au]
            setup["period_in"] = period_in
            setup["Rcap_in"] = Rcap_in

        setup["period"] = period
        setup["Rcap"] = Rcap
    else:
        setup["sma_ini"] = 0.

    return setup


In [ ]:

def gen_data(image_number: int):
    return {
        "git hash": "8743b52063cd84097a65d1633f5c74f5",
        "resolution": random.choice([64, 128, 256, 512]),
        "time": random.uniform(2000.0, 50000.0),
        "path to image": f"/path/to/image_{image_number}.png",
        "eccentricity": random.uniform(0.0, 1.0),
        "semi-major axis": random.uniform(1.0, 10.0),
        "primary mass": random.uniform(1000.0, 100000.0),
        "primary racc": random.uniform(1.0, 10.0),
        "primary Teff": random.uniform(100.0, 1000.0),
        "primary Reff": random.uniform(1.0, 5.0),
        "secondary mass": random.uniform(10.0, 1000.0),
        "secondary racc": random.uniform(0.0, 1.0),
        "secondary Teff": random.uniform(1.0, 100.0),
        "secondary Reff": random.uniform(0.0, 1.0),
        "adiabatic index": random.uniform(1.0, 10.0),
        "mean molecular weight": random.uniform(1.0, 50.0),
        "icooling": int(random.uniform(20.0, 50.0)),
    }